In [1]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00


In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [3]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 3
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem.
It is important that you do this by following these steps:

1. Define parameters and variables
2. Define the objective function
3. Define the constraints

Please note that each answer is always dedicated to one step. Therefore, do not provide extra information than what is asked. The answers will be manually combined by the user to create the full model."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
USER_PROMPT_1 = """We are now at the first step."""
USER_PROMPT_2 = """We are now at the second step."""
USER_PROMPT_3 = """We are now at the third step."""
PROBLEM_DESCRIPTION = """We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number.

We want to maximize the daily profit of this alkylation process.
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup.

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas.
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed.
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate.
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup."""

client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 12:04:37.335156


## Step 1 - Generate Mathematical Formulation

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1)
]

response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

Step 1: Define parameters and variables

Parameters:
- $C_{olefin}$: cost per barrel of olefin feed
- $C_{isobutane}$: cost per barrel of isobutane recycle
- $C_{acid}$: cost per thousand pounds of acid addition
- $C_{makeup}$: cost per barrel of isobutane makeup
- $P_{alkylate}$: revenue per barrel of alkylate yield

Variables:
- $x_1$: olefin feed (barrels per day)
- $x_2$: isobutane recycle (barrels per day)
- $x_3$: acid addition rate (thousands of pounds per day)
- $x_4$: alkylate yield (barrels per day)
- $x_5$: isobutane makeup (barrels per day)
- $x_6$: acid strength (weight per cent)
- $x_7$: motor octane number
- $x_8$: external isobutane-to-olefin ratio
- $x_9$: acid dilution factor
- $x_{10}$: F-4 performance number

The variables $x_1$ through $x_5$ are decision variables that can be adjusted to optimize the objective function, while $x_6$ through $x_{10}$ are dependent variables that are determined based on the given regression formulas.

In [6]:
print(response_1_text)

Step 1: Define parameters and variables

Parameters:
- $C_{olefin}$: cost per barrel of olefin feed
- $C_{isobutane}$: cost per barrel of isobutane recycle
- $C_{acid}$: cost per thousand pounds of acid addition
- $C_{makeup}$: cost per barrel of isobutane makeup
- $P_{alkylate}$: revenue per barrel of alkylate yield

Variables:
- $x_1$: olefin feed (barrels per day)
- $x_2$: isobutane recycle (barrels per day)
- $x_3$: acid addition rate (thousands of pounds per day)
- $x_4$: alkylate yield (barrels per day)
- $x_5$: isobutane makeup (barrels per day)
- $x_6$: acid strength (weight per cent)
- $x_7$: motor octane number
- $x_8$: external isobutane-to-olefin ratio
- $x_9$: acid dilution factor
- $x_{10}$: F-4 performance number

The variables $x_1$ through $x_5$ are decision variables that can be adjusted to optimize the objective function, while $x_6$ through $x_{10}$ are dependent variables that are determined based on the given regression formulas.


In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1),
    ChatMessage(role="assistant", content=response_1_text),
    ChatMessage(role="user", content=USER_PROMPT_2),
]

response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [8]:
Markdown(response_2_text)

Step 2: Define the objective function

The objective function for this problem is to maximize daily profit, which is defined as the revenue generated from alkylate yield multiplied by the motor octane number, minus the operational costs (olefin feed, isobutane recycle, acid addition rate, and isobutane makeup). Mathematically, the objective function can be expressed as:

$$
\text{Maximize } f(x) = P_{alkylate} \cdot x_4 \cdot x_7 - (C_{olefin} \cdot x_1 + C_{isobutane} \cdot x_2 + C_{acid} \cdot x_3 + C_{makeup} \cdot x_5)
$$

This objective function represents the daily profit of the alkylation process, where $P_{alkylate}$ is the revenue per barrel of alkylate yield, $x_4$ is the alkylate yield (barrels per day), $x_7$ is the motor octane number, and the coefficients $C_{olefin}$, $C_{isobutane}$, $C_{acid}$, and $C_{makeup}$ represent the costs per unit for olefin feed, isobutane recycle, acid addition rate, and isobutane makeup, respectively.

In [9]:
print(response_2_text)

Step 2: Define the objective function

The objective function for this problem is to maximize daily profit, which is defined as the revenue generated from alkylate yield multiplied by the motor octane number, minus the operational costs (olefin feed, isobutane recycle, acid addition rate, and isobutane makeup). Mathematically, the objective function can be expressed as:

$$
\text{Maximize } f(x) = P_{alkylate} \cdot x_4 \cdot x_7 - (C_{olefin} \cdot x_1 + C_{isobutane} \cdot x_2 + C_{acid} \cdot x_3 + C_{makeup} \cdot x_5)
$$

This objective function represents the daily profit of the alkylation process, where $P_{alkylate}$ is the revenue per barrel of alkylate yield, $x_4$ is the alkylate yield (barrels per day), $x_7$ is the motor octane number, and the coefficients $C_{olefin}$, $C_{isobutane}$, $C_{acid}$, and $C_{makeup}$ represent the costs per unit for olefin feed, isobutane recycle, acid addition rate, and isobutane makeup, respectively.


In [10]:
messages_3 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1),
    ChatMessage(role="assistant", content=response_1_text),
    ChatMessage(role="user", content=USER_PROMPT_2),
    ChatMessage(role="assistant", content=response_2_text),
    ChatMessage(role="user", content=USER_PROMPT_3),
]

response_3 = client.chat(
    model=MODEL_ID,
    messages=messages_3,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_3_text = response_3.choices[0].message.content

In [11]:
Markdown(response_3_text)

Step 3: Define the constraints

The following constraints are imposed by the nature of the chemical process:

1. External isobutane-to-olefin ratio:
\[x_8 = \frac{x_2 + x_5}{x_1}\]

2. Acid strength:
\[x_6 = \frac{93000 \cdot x_3}{x_3 \cdot x_9 + 1000 \cdot x_3}\]

3. Alkylate yield:
\[x_4 = 1.12 \cdot x_1 + 0.13167 \cdot x_8 \cdot x_1 - 0.00667 \cdot x_8^2 \cdot x_1\]

4. Motor octane number:
\[x_7 = 86.35 + 1.098 \cdot x_8 - 0.038 \cdot x_8^2 + 0.325 \cdot (x_6 - 89)\]

5. Acid dilution factor:
\[x_9 = 35.82 - 0.222 \cdot x_{10}\]

6. F-4 performance number:
\[x_{10} = -133 + 3 \cdot x_7\]

7. Lower and upper bounds for each variable:
\[l_i \leq x_i \leq u_i \text{ for } i = 1, 2, \ldots, 10\]

8. Combined olefin feed and isobutane makeup:
\[1.22 \cdot x_4 = x_1 + x_5\]

These constraints ensure that the chemical process adheres to the physical limitations and relationships between the variables. The lower and upper bounds for each variable ($l_i$ and $u_i$) are given based on the specific process requirements.

In [12]:
print(response_3_text)

Step 3: Define the constraints

The following constraints are imposed by the nature of the chemical process:

1. External isobutane-to-olefin ratio:
\[x_8 = \frac{x_2 + x_5}{x_1}\]

2. Acid strength:
\[x_6 = \frac{93000 \cdot x_3}{x_3 \cdot x_9 + 1000 \cdot x_3}\]

3. Alkylate yield:
\[x_4 = 1.12 \cdot x_1 + 0.13167 \cdot x_8 \cdot x_1 - 0.00667 \cdot x_8^2 \cdot x_1\]

4. Motor octane number:
\[x_7 = 86.35 + 1.098 \cdot x_8 - 0.038 \cdot x_8^2 + 0.325 \cdot (x_6 - 89)\]

5. Acid dilution factor:
\[x_9 = 35.82 - 0.222 \cdot x_{10}\]

6. F-4 performance number:
\[x_{10} = -133 + 3 \cdot x_7\]

7. Lower and upper bounds for each variable:
\[l_i \leq x_i \leq u_i \text{ for } i = 1, 2, \ldots, 10\]

8. Combined olefin feed and isobutane makeup:
\[1.22 \cdot x_4 = x_1 + x_5\]

These constraints ensure that the chemical process adheres to the physical limitations and relationships between the variables. The lower and upper bounds for each variable ($l_i$ and $u_i$) are given based on the sp

In [13]:
response_text = '\n'.join([response_1_text, response_2_text, response_3_text])

Markdown(response_text)

Step 1: Define parameters and variables

Parameters:
- $C_{olefin}$: cost per barrel of olefin feed
- $C_{isobutane}$: cost per barrel of isobutane recycle
- $C_{acid}$: cost per thousand pounds of acid addition
- $C_{makeup}$: cost per barrel of isobutane makeup
- $P_{alkylate}$: revenue per barrel of alkylate yield

Variables:
- $x_1$: olefin feed (barrels per day)
- $x_2$: isobutane recycle (barrels per day)
- $x_3$: acid addition rate (thousands of pounds per day)
- $x_4$: alkylate yield (barrels per day)
- $x_5$: isobutane makeup (barrels per day)
- $x_6$: acid strength (weight per cent)
- $x_7$: motor octane number
- $x_8$: external isobutane-to-olefin ratio
- $x_9$: acid dilution factor
- $x_{10}$: F-4 performance number

The variables $x_1$ through $x_5$ are decision variables that can be adjusted to optimize the objective function, while $x_6$ through $x_{10}$ are dependent variables that are determined based on the given regression formulas.
Step 2: Define the objective function

The objective function for this problem is to maximize daily profit, which is defined as the revenue generated from alkylate yield multiplied by the motor octane number, minus the operational costs (olefin feed, isobutane recycle, acid addition rate, and isobutane makeup). Mathematically, the objective function can be expressed as:

$$
\text{Maximize } f(x) = P_{alkylate} \cdot x_4 \cdot x_7 - (C_{olefin} \cdot x_1 + C_{isobutane} \cdot x_2 + C_{acid} \cdot x_3 + C_{makeup} \cdot x_5)
$$

This objective function represents the daily profit of the alkylation process, where $P_{alkylate}$ is the revenue per barrel of alkylate yield, $x_4$ is the alkylate yield (barrels per day), $x_7$ is the motor octane number, and the coefficients $C_{olefin}$, $C_{isobutane}$, $C_{acid}$, and $C_{makeup}$ represent the costs per unit for olefin feed, isobutane recycle, acid addition rate, and isobutane makeup, respectively.
Step 3: Define the constraints

The following constraints are imposed by the nature of the chemical process:

1. External isobutane-to-olefin ratio:
\[x_8 = \frac{x_2 + x_5}{x_1}\]

2. Acid strength:
\[x_6 = \frac{93000 \cdot x_3}{x_3 \cdot x_9 + 1000 \cdot x_3}\]

3. Alkylate yield:
\[x_4 = 1.12 \cdot x_1 + 0.13167 \cdot x_8 \cdot x_1 - 0.00667 \cdot x_8^2 \cdot x_1\]

4. Motor octane number:
\[x_7 = 86.35 + 1.098 \cdot x_8 - 0.038 \cdot x_8^2 + 0.325 \cdot (x_6 - 89)\]

5. Acid dilution factor:
\[x_9 = 35.82 - 0.222 \cdot x_{10}\]

6. F-4 performance number:
\[x_{10} = -133 + 3 \cdot x_7\]

7. Lower and upper bounds for each variable:
\[l_i \leq x_i \leq u_i \text{ for } i = 1, 2, \ldots, 10\]

8. Combined olefin feed and isobutane makeup:
\[1.22 \cdot x_4 = x_1 + x_5\]

These constraints ensure that the chemical process adheres to the physical limitations and relationships between the variables. The lower and upper bounds for each variable ($l_i$ and $u_i$) are given based on the specific process requirements.

In [14]:
print(response_text)

Step 1: Define parameters and variables

Parameters:
- $C_{olefin}$: cost per barrel of olefin feed
- $C_{isobutane}$: cost per barrel of isobutane recycle
- $C_{acid}$: cost per thousand pounds of acid addition
- $C_{makeup}$: cost per barrel of isobutane makeup
- $P_{alkylate}$: revenue per barrel of alkylate yield

Variables:
- $x_1$: olefin feed (barrels per day)
- $x_2$: isobutane recycle (barrels per day)
- $x_3$: acid addition rate (thousands of pounds per day)
- $x_4$: alkylate yield (barrels per day)
- $x_5$: isobutane makeup (barrels per day)
- $x_6$: acid strength (weight per cent)
- $x_7$: motor octane number
- $x_8$: external isobutane-to-olefin ratio
- $x_9$: acid dilution factor
- $x_{10}$: F-4 performance number

The variables $x_1$ through $x_5$ are decision variables that can be adjusted to optimize the objective function, while $x_6$ through $x_{10}$ are dependent variables that are determined based on the given regression formulas.
Step 2: Define the objective funct

## Step 2 - Generate the Pyomo Code

In [15]:
messages_4 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_text)
]

In [16]:
response_4 = client.chat(
    model=MODEL_ID,
    messages=messages_4,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_4_text = response_4.choices[0].message.content

In [17]:
Markdown(response_4_text)

Here is the Python code using Pyomo to solve the optimization problem:

```python
import pyomo.environ as pyo

# Define the model
model = pyo.AbstractModel()

# Define the parameters
model.C_olefin = pyo.Param()
model.C_isobutane = pyo.Param()
model.C_acid = pyo.Param()
model.C_makeup = pyo.Param()
model.P_alkylate = pyo.Param()

# Define the variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals)
model.x2 = pyo.Var(within=pyo.NonNegativeReals)
model.x3 = pyo.Var(within=pyo.NonNegativeReals)
model.x4 = pyo.Var(within=pyo.NonNegativeReals)
model.x5 = pyo.Var(within=pyo.NonNegativeReals)
model.x6 = pyo.Var(within=pyo.NonNegativeReals)
model.x7 = pyo.Var(within=pyo.NonNegativeReals)
model.x8 = pyo.Var(within=pyo.NonNegativeReals)
model.x9 = pyo.Var(within=pyo.NonNegativeReals)
model.x10 = pyo.Var(within=pyo.NonNegativeReals)

# Define the objective function
def obj_func(model):
    return model.P_alkylate * model.x4 * model.x7 - (model.C_olefin * model.x1 + model.C_isobutane * model.x2 + model.C_acid * model.x3 + model.C_makeup * model.x5)
model.obj = pyo.Objective(rule=obj_func, sense=pyo.maximize)

# Define the constraints
def constraint1(model):
    return model.x8 == (model.x2 + model.x5) / model.x1
model.con1 = pyo.Constraint(rule=constraint1)

def constraint2(model):
    return model.x6 == (93000 * model.x3) / (model.x3 * model.x9 + 1000 * model.x3)
model.con2 = pyo.Constraint(rule=constraint2)

def constraint3(model):
    return model.x4 == 1.12 * model.x1 + 0.13167 * model.x8 * model.x1 - 0.00667 * model.x8**2 * model.x1
model.con3 = pyo.Constraint(rule=constraint3)

def constraint4(model):
    return model.x7 == 86.35 + 1.098 * model.x8 - 0.038 * model.x8**2 + 0.325 * (model.x6 - 89)
model.con4 = pyo.Constraint(rule=constraint4)

def constraint5(model):
    return model.x9 == 35.82 - 0.222 * model.x10
model.con5 = pyo.Constraint(rule=constraint5)

def constraint6(model):
    return model.x10 == -133 + 3 * model.x7
model.con6 = pyo.Constraint(rule=constraint6)

# Define the lower and upper bounds for each variable
# Replace these with your actual bounds
model.x1.setlb(0)
model.x1.setub(1000)
model.x2.setlb(0)
model.x2.setub(1000)
model.x3.setlb(0)
model.x3.setub(1000)
model.x4.setlb(0)
model.x4.setub(1000)
model.x5.setlb(0)
model.x5.setub(1000)
model.x6.setlb(0)
model.x6.setub(100)
model.x7.setlb(0)
model.x7.setub(100)
model.x8.setlb(0)
model.x8.setub(100)
model.x9.setlb(0)
model.x9.setub(100)
model.x10.setlb(0)
model.x10.setub(100)

# Define the combined olefin feed and isobutane makeup constraint
def constraint7(model):
    return 1.22 * model.x4 == model.x1 + model.x5
model.con7 = pyo.Constraint(rule=constraint7)

# Create a concrete model using sample data
sample_data = {None: {'C_olefin': 10, 'C_isobutane': 20, 'C_acid': 30, 'C_makeup': 40, 'P_alkylate': 50}}
instance = model.create_instance(sample_data)

# Solve the optimization problem
opt = pyo.SolverFactory('glpk')
results = opt.solve(instance)
results.write()

# Print the results
print("Optimal Daily Profit:", pyo.value(instance.obj))
print("Optimal Variable Values:")
print("x1:", pyo.value(instance.x1))
print("x2:", pyo.value(instance.x2))
print("x3:", pyo.value(instance.x3))
print("x4:", pyo.value(instance.x4))
print("x5:", pyo.value(instance.x5))
print("x6:", pyo.value(instance.x6))
print("x7:", pyo.value(instance.x7))
print("x8:", pyo.value(instance.x8))
print("x9:", pyo.value(instance.x9))
print("x10:", pyo.value(instance.x10))
```

In [18]:
print(response_4_text)

Here is the Python code using Pyomo to solve the optimization problem:

```python
import pyomo.environ as pyo

# Define the model
model = pyo.AbstractModel()

# Define the parameters
model.C_olefin = pyo.Param()
model.C_isobutane = pyo.Param()
model.C_acid = pyo.Param()
model.C_makeup = pyo.Param()
model.P_alkylate = pyo.Param()

# Define the variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals)
model.x2 = pyo.Var(within=pyo.NonNegativeReals)
model.x3 = pyo.Var(within=pyo.NonNegativeReals)
model.x4 = pyo.Var(within=pyo.NonNegativeReals)
model.x5 = pyo.Var(within=pyo.NonNegativeReals)
model.x6 = pyo.Var(within=pyo.NonNegativeReals)
model.x7 = pyo.Var(within=pyo.NonNegativeReals)
model.x8 = pyo.Var(within=pyo.NonNegativeReals)
model.x9 = pyo.Var(within=pyo.NonNegativeReals)
model.x10 = pyo.Var(within=pyo.NonNegativeReals)

# Define the objective function
def obj_func(model):
    return model.P_alkylate * model.x4 * model.x7 - (model.C_olefin * model.x1 + model.C_isobutane * model.x2 

### Code Executability

In [20]:
import pyomo.environ as pyo

# Define the model
model = pyo.AbstractModel()

# Define the parameters
model.C_olefin = pyo.Param()
model.C_isobutane = pyo.Param()
model.C_acid = pyo.Param()
model.C_makeup = pyo.Param()
model.P_alkylate = pyo.Param()

# Define the variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals)
model.x2 = pyo.Var(within=pyo.NonNegativeReals)
model.x3 = pyo.Var(within=pyo.NonNegativeReals)
model.x4 = pyo.Var(within=pyo.NonNegativeReals)
model.x5 = pyo.Var(within=pyo.NonNegativeReals)
model.x6 = pyo.Var(within=pyo.NonNegativeReals)
model.x7 = pyo.Var(within=pyo.NonNegativeReals)
model.x8 = pyo.Var(within=pyo.NonNegativeReals)
model.x9 = pyo.Var(within=pyo.NonNegativeReals)
model.x10 = pyo.Var(within=pyo.NonNegativeReals)

# Define the objective function
def obj_func(model):
    return model.P_alkylate * model.x4 * model.x7 - (model.C_olefin * model.x1 + model.C_isobutane * model.x2 + model.C_acid * model.x3 + model.C_makeup * model.x5)
model.obj = pyo.Objective(rule=obj_func, sense=pyo.maximize)

# Define the constraints
def constraint1(model):
    return model.x8 == (model.x2 + model.x5) / model.x1
model.con1 = pyo.Constraint(rule=constraint1)

def constraint2(model):
    return model.x6 == (93000 * model.x3) / (model.x3 * model.x9 + 1000 * model.x3)
model.con2 = pyo.Constraint(rule=constraint2)

def constraint3(model):
    return model.x4 == 1.12 * model.x1 + 0.13167 * model.x8 * model.x1 - 0.00667 * model.x8**2 * model.x1
model.con3 = pyo.Constraint(rule=constraint3)

def constraint4(model):
    return model.x7 == 86.35 + 1.098 * model.x8 - 0.038 * model.x8**2 + 0.325 * (model.x6 - 89)
model.con4 = pyo.Constraint(rule=constraint4)

def constraint5(model):
    return model.x9 == 35.82 - 0.222 * model.x10
model.con5 = pyo.Constraint(rule=constraint5)

def constraint6(model):
    return model.x10 == -133 + 3 * model.x7
model.con6 = pyo.Constraint(rule=constraint6)

# Define the lower and upper bounds for each variable
# Replace these with your actual bounds
model.x1.setlb(0)
model.x1.setub(1000)
model.x2.setlb(0)
model.x2.setub(1000)
model.x3.setlb(0)
model.x3.setub(1000)
model.x4.setlb(0)
model.x4.setub(1000)
model.x5.setlb(0)
model.x5.setub(1000)
model.x6.setlb(0)
model.x6.setub(100)
model.x7.setlb(0)
model.x7.setub(100)
model.x8.setlb(0)
model.x8.setub(100)
model.x9.setlb(0)
model.x9.setub(100)
model.x10.setlb(0)
model.x10.setub(100)

# Define the combined olefin feed and isobutane makeup constraint
def constraint7(model):
    return 1.22 * model.x4 == model.x1 + model.x5
model.con7 = pyo.Constraint(rule=constraint7)

# Create a concrete model using sample data
sample_data = {None: {'C_olefin': 10, 'C_isobutane': 20, 'C_acid': 30, 'C_makeup': 40, 'P_alkylate': 50}}
instance = model.create_instance(sample_data)

# Solve the optimization problem
opt = pyo.SolverFactory('ipopt')
results = opt.solve(instance)
results.write()

# Print the results
print("Optimal Daily Profit:", pyo.value(instance.obj))
print("Optimal Variable Values:")
print("x1:", pyo.value(instance.x1))
print("x2:", pyo.value(instance.x2))
print("x3:", pyo.value(instance.x3))
print("x4:", pyo.value(instance.x4))
print("x5:", pyo.value(instance.x5))
print("x6:", pyo.value(instance.x6))
print("x7:", pyo.value(instance.x7))
print("x8:", pyo.value(instance.x8))
print("x9:", pyo.value(instance.x9))
print("x10:", pyo.value(instance.x10))

RuntimeError: Cannot access 'setlb' on AbstractScalarVar 'x1' before it has been constructed (initialized): 'setlb' is an attribute on an Abstract component and cannot be accessed until the component has been fully constructed (converted to a Concrete component) using AbstractModel.create_instance() or x1.construct().

### Solution Correctness

In [23]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
model.C_olefin = 5.04
model.C_isobutane = 0.035
model.C_acid = 10
model.C_makeup = 3.36
model.P_alkylate = 0.63

# Define the variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals)
model.x2 = pyo.Var(within=pyo.NonNegativeReals)
model.x3 = pyo.Var(within=pyo.NonNegativeReals)
model.x4 = pyo.Var(within=pyo.NonNegativeReals)
model.x5 = pyo.Var(within=pyo.NonNegativeReals)
model.x6 = pyo.Var(within=pyo.NonNegativeReals)
model.x7 = pyo.Var(within=pyo.NonNegativeReals)
model.x8 = pyo.Var(within=pyo.NonNegativeReals)
model.x9 = pyo.Var(within=pyo.NonNegativeReals)
model.x10 = pyo.Var(within=pyo.NonNegativeReals)

# Define the objective function
def obj_func(model):
    return model.P_alkylate * model.x4 * model.x7 - (model.C_olefin * model.x1 + model.C_isobutane * model.x2 + model.C_acid * model.x3 + model.C_makeup * model.x5)
model.obj = pyo.Objective(rule=obj_func, sense=pyo.maximize)

# Define the constraints
def constraint1(model):
    return model.x8 == (model.x2 + model.x5) / model.x1
model.con1 = pyo.Constraint(rule=constraint1)

def constraint2(model):
    return model.x6 == (93000 * model.x3) / (model.x3 * model.x9 + 1000 * model.x3)
model.con2 = pyo.Constraint(rule=constraint2)

def constraint3(model):
    return model.x4 == 1.12 * model.x1 + 0.13167 * model.x8 * model.x1 - 0.00667 * model.x8**2 * model.x1
model.con3 = pyo.Constraint(rule=constraint3)

def constraint4(model):
    return model.x7 == 86.35 + 1.098 * model.x8 - 0.038 * model.x8**2 + 0.325 * (model.x6 - 89)
model.con4 = pyo.Constraint(rule=constraint4)

def constraint5(model):
    return model.x9 == 35.82 - 0.222 * model.x10
model.con5 = pyo.Constraint(rule=constraint5)

def constraint6(model):
    return model.x10 == -133 + 3 * model.x7
model.con6 = pyo.Constraint(rule=constraint6)

# Define the lower and upper bounds for each variable
# Replace these with your actual bounds
model.x1.setlb(0)
model.x1.setub(2000)
model.x2.setlb(0)
model.x2.setub(16000)
model.x3.setlb(0)
model.x3.setub(120)
model.x4.setlb(0)
model.x4.setub(5000)
model.x5.setlb(0)
model.x5.setub(2000)
model.x6.setlb(85)
model.x6.setub(93)
model.x7.setlb(90)
model.x7.setub(95)
model.x8.setlb(3)
model.x8.setub(12)
model.x9.setlb(1.2)
model.x9.setub(4)
model.x10.setlb(145)
model.x10.setub(162)

# Define the combined olefin feed and isobutane makeup constraint
def constraint7(model):
    return 1.22 * model.x4 == model.x1 + model.x5
model.con7 = pyo.Constraint(rule=constraint7)

# Create a concrete model using sample data
sample_data = {None: {'C_olefin': 10, 'C_isobutane': 20, 'C_acid': 30, 'C_makeup': 40, 'P_alkylate': 50}}
instance = model.create_instance(sample_data)

# Solve the optimization problem
opt = pyo.SolverFactory('ipopt')
results = opt.solve(instance)
results.write()

# Print the results
print("Optimal Daily Profit:", pyo.value(instance.obj))
print("Optimal Variable Values:")
print("x1:", pyo.value(instance.x1))
print("x2:", pyo.value(instance.x2))
print("x3:", pyo.value(instance.x3))
print("x4:", pyo.value(instance.x4))
print("x5:", pyo.value(instance.x5))
print("x6:", pyo.value(instance.x6))
print("x7:", pyo.value(instance.x7))
print("x8:", pyo.value(instance.x8))
print("x9:", pyo.value(instance.x9))
print("x10:", pyo.value(instance.x10))

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 10
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.03218197822570801
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
Optima